In [72]:
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
import requests
import psycopg2
import boto3
import json
import os
import re

# Get environment variable
load_dotenv('.env')
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

#initialize S3
S3 = boto3.client('s3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

#initialize dynamodb
dynamodb = boto3.resource('dynamodb', region_name='ap-southeast-2',
  aws_access_key_id=AWS_ACCESS_KEY_ID,
  aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

table_recipe = dynamodb.Table('FoodRecipes')

conn = psycopg2.connect(
    "dbname=eltrial user=top password=1234"
)

In [60]:
BUCKET = "eltrial"
OBJECT = "SCRAPING/SIMPLE_EXTRACTED/"
FILENAME = [
    "Mthai-recipe-2020-07-22_20:25:58.json",
    "Kapook-recipe-2020-07-25_15:25:52.json",
    "Wongnai-recipe-2020-07-25_13:54:26.json"
]

for f in FILENAME:
    S3.download_file(BUCKET, f"{OBJECT}{f}", f)

In [65]:
with open("Wongnai-recipe-2020-07-25_13:54:26.json") as f:
    wongnai = json.load(f)
    
with open("Kapook-recipe-2020-07-25_15:25:52.json") as f:
    kapook = json.load(f)
    
with open("Mthai-recipe-2020-07-22_20:25:58.json") as f:
    mthai = json.load(f)

In [124]:
num = re.compile("\d+ กรัม$")

In [130]:
num.search(wongnai[j][k]).string

'150 กรัม'

# Wongnai

In [574]:
units = {
    "กรัม": "g",
    "g": "g",
    "grams": "g",
    "กระม": "g",
    "กิโล": "kg",
    "ฟอง": "unit",
    "ชิ้นใหญ่": "unknown",
    "มิลลิลิตร": "ml",
    "ก้อน": "unknown",
    "ถ้วย": "unknown",
    "ช้อนโต๊ะ": (15, "ml"),
    "ชิ้น.*": "unknown",
    "กร้ม": "g",
    "ml": "ml",
    "kg": (1000, "g"),
    "กก": (1000, "g"),
    "ขีด": (100, "g"),
    "กิโลกรัม": (1000, "g"),
    "กรม": "g",
    "ท่อน": "unknown",
    "โครง": "unknown",
    "ใบ": "unit",
    "แพ็ค": "unknown",
    "mg": (0.001, "g"),
    "ก$": "g",
    "เส้น": "unknown",
    "น่อง": "unknown",
    "ปีก": "unknown",
    "ขา": "unknown",
    "วง": "unknown",
    "ตัว": "unknown",
    "ขึด": "unknown",
    "ซอง": "unknown",
    "อก": "unknown",
    "จาน": "unknown",
    "โล": (1000, "g"),
    "แท่ง": "unknown",
    "บาท": "baht",
    "ไม้": "unknown",
    "ดุ้นสั้น": "unknown",
    "ตีน": "unknown",
    "Kg": (1000, "g"),
    "ทัพพี": "unknown",
    "ฟอว": "unit",
    "ถาด": "unknown",
    "ช้ินโต๊ะ": (15, "g"),
    "ห่อ": "unknown",
    "ลูก": "unit",
    "KG": (1000, "g"),
    "แผ่น.*": "unknown",
    "G": "g",
    "ตามสะดวก": "unknown",
    "ชต": (15, "g"),
    "pack": "unknown",
    "พวง": "unknown",
    "ถุง": "unknown",
    "pcs": "unit",
    "สไลด์": "unknown",
    "โคร": "unknown",
    "ฟ$": "unit",
    "ชาม": "unknown",
    "แพค": "unknown",
    "ขีเ": (100, "g"),
    "ช้อน$": "unknown",
    "อัน": "unknown",
    "แพ๊ค": "unknown",
    "ครึ่งโล": (500, "g"),
    "หม้อ": "unknown",
    "กล่อง": "unknown",
    "กรีม": "g",
    "กนัม": "g",
    "สะโพก": "unknown",
    "ซีก": "unknown",
    "ขรด": "unknown",
    "ชีด": "unknown",
    "หยิบมือ": "unknown",
    "โคลง": "unknown",
    "มือ": "unknown",
    "ถต": (140, "g"),
    "กรับ": "g",
    "แ$": "g",
    "กม": "g",
    "ชุด": "unknown",
    "ช้อนโตณะ": (15, "g"),
    "หอง": "unit",
    "หาง": "unknown",
    "ช้อนโตํะ": "unknown",
    "ที่": "unknown",
}


patterns = [
    "\d+ - \d+\ *{}",
    "\d+-\d+\ *{}",
    "\d+\ *{}",
    "\d+\t*{}",
]

regex_patterns = []
for u in units:
    for p in patterns:
        regex_patterns.append(p.format(u))
        
regex_pattern = re.compile("|".join(regex_patterns))

In [575]:
ig

'ครึ่1/2'

In [577]:
includes = re.compile("(หมู|ไก่|วัว|^ไข่$|^ไข่ไก่|$นม)")
excludes = re.compile("นมข้นหวาน|ผง|คนอร์|มาม่า|คาร์เนชั่น|หมูกรอบ|กระดูก|ปู|^ซอส|^เครื่องเทศ|^เต้าหู้|อินทผาลัม|ลูกชิ้น|หมูแฮม|ไข่แดงเค็ม|น้ำมัน|คอร์|ไส้กรอก|หัวใจ|กระเพาะ|ปอด|รสดี|คนอ|น้ำซุป|หมูยอ|น้ำ|ซีอิ๊ว|ชุด|^พริกไทย|ซีอิ้ว|สต๊อก|แป้ง|^พริก|เลือด|ชุปก้อน")
excludes_units = re.compile("สำหรับ|ครึ่1/2")
num = re.compile("\d")
no_unit = re.compile("^\d+$|^\d+\ *-\d\ *$|^\d+/\d+$")

units = []
l = False
f = 0
for j in wongnai:
#     print(f)
    for k in wongnai[j]:
        _k = k.split(" ")[0]
        if includes.search(_k) and not excludes.search(k):
            ig = wongnai[j][k].strip().replace("\u200b", "").replace(".", "")
            if num.search(ig) and not regex_pattern.search(ig) and not no_unit.search(ig) and not excludes_units.search(ig):
                l = True
                print(ig)
                print(k, ig)
                break
                
    f += 1
    if l:
        break
#         re.sub(num, "", wongnai[j][k])

print(f)

9221


In [115]:
units = []
for u in _units:
    units.extend(u)

In [116]:
dfunits = pd.DataFrame(units)

In [117]:
unq = pd.unique(dfunits.values.flatten())

In [118]:
unq.tolist()

['กระป๋อง',
 'เม็ด',
 'ฟอง',
 'เล็กน้อย',
 'พอประมาณ',
 'ห่อ',
 'กรัม',
 'ช้อนโต๊ะ',
 'ช้อนชา',
 'ถ้วยตวง',
 'ก้อน',
 'ชิ้นใหญ่',
 'ต้น',
 'ตามชอบ',
 'มิลลิลิตร',
 '',
 'แพ็ค',
 'ใบ',
 'ครึ่งช้อนโต๊ะ',
 'ครึ่งช้อนชา',
 'นิดหน่อย',
 '+ถ้วย',
 'ถ้วย',
 'กิโลกรัม',
 'ลิตร',
 'ตัว',
 'ถุง',
 'ชต',
 'โล',
 'ชช',
 'ถต',
 'ลูก',
 'ออนซ์',
 'เต็มแก้ว',
 'กำ',
 'กลีบ',
 'หัว',
 'ราก',
 'ส่วน',
 'ซอง',
 'แล้วแต่ชอบ',
 'มล',
 'ครึ่งกก',
 'เหยาะ',
 'เหยาะๆลงไป',
 'ชิ้น',
 'กีบ',
 'กก',
 'กำมือ',
 'ครึ่งถ้วยตวง',
 'ดอก',
 'ซีก',
 'ฝัก',
 'ถ้วน',
 'กระป๋อง(กรัม)',
 'ทัพพี',
 'กะตามชอบให้ข้าวลักษณะใด',
 'แง่ง(กรัม)',
 'กร้ม',
 'ก้อนเล็ก',
 'ml',
 'กรัม(ก่อนแช่)',
 'ครึ่งหัว',
 'ขีด',
 'ตามใจชอบ',
 'สำหรับผัด',
 'สำหรับปรุงรส',
 'สำหรับตกแต่ง',
 '+ชต',
 'แคปซูล',
 'ชิ้น(g)',
 'ชิ้น(ใหญ่)',
 'ชุด',
 'ปั้น',
 'ขวด',
 'ชน',
 'ช้อนช้า',
 'ช็อต',
 'แก้ว',
 '|ช้อนชา',
 'ครึ่งลิตร',
 'g',
 'ครึ่งตัว',
 'kg',
 'ขีดมล',
 'แผ่น',
 'กิโล',
 'เส้น',
 'เเว่น',
 'ช้อน',
 'ชาม',
 'ครึ่งลูก',
 'ทาขนมปัง',
 'พวง',
 'ก

# Insert Wongnai

In [70]:
# cursor = conn.cursor()
# for j in wongnai:
#     res = cursor.execute("INSERT INTO ingredients (menu_name, ingredient_json) VALUES(%s, %s)", 
#                (j, json.dumps(wongnai[j])))
# conn.commit()
# cursor.close()

In [71]:
kapook

{'ไมโครเวฟพร้อมจ้า ! ชวนทำไดฟูกุสตรอว์เบอร์รี เมนูไมโครเวฟ แป้งสีสวยเหนียวนุ่ม สอดไส้ถั่วแดงกวนกับผลไม้สด ปั้นลูกโตน่าหม่ำ จับแช่เย็นยิ่งฟิน_https://cooking.kapook.com/view206822.html': ['แป้งข้าวเหนียว 1+1/2 ถ้วย',
  'แป้งข้าวโพด 1 ช้อนโต๊ะ',
  'น้ำตาลทรายทรายป่นละเอียด 5 ช้อนโต๊ะ',
  'น้ำสะอาด 250 กรัม',
  'สีผสมอาหารสีแดง (ใส่หรือไม่ใส่ก็ได้) \xa0',
  'แป้งมันสำปะหลัง สำหรับทำแป้งนวล 100 กรัม (นำไปผัดด้วยไฟกลางค่อนข้างอ่อน 5 นาที)\n\t\xa0',
  'สตรอว์เบอร์รี\n\t\xa0',
  'เนยสดเค็ม 1/2 ช้อนโต๊ะ',
  'ถั่วแดงกระป๋อง 200 กรัม',
  'น้ำตาลทรายแดง 160 กรัม\n\t\xa0'],
 ' ขนมหวานวันหยุด เมนูคัพเค้กไมโครเวฟ เมนูไมโครเวฟ อร่อยนุ่มนานแต่งหน้าตามชอบ ทำง่ายไม่ต้องใช้เตาอบเหมาะกับเด็กหอ\xa0\xa0_https://cooking.kapook.com/view202667.html': ['เนยสด 120 กรัม',
  'น้ำตาลทราย 120 กรัม',
  'กลิ่นวานิลลา 1/2 ช้อนชา',
  'ไข่ 2 ฟอง',
  'แป้งสาลีอเนกประสงค์ 120 กรัม',
  'ครีมและเกล็ดน้ำตาลตกแต่งตามชอบ\n\t\xa0'],
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 สะบัดบ๊อบให้เมนูช็อกโกแลตบอลสุดเชย แล้วมาลองทำช็อกโกแล

In [46]:
# cursor = conn.cursor()
# cursor.execute("SELECT * FROM ingredients;")
# a = cursor.fetchall() 
# # cursor.close()

In [607]:
import requests
res = requests.get("http://localhost:8000?keyword=หมูสับ").text
jl = json.loads(res)

In [608]:
with open("test.json", "w+") as f:
    json.dump(jl, f)

In [609]:
jl["menu_prices"]

{'1 แกงจืดตำลึงเต้าหู้หมูสับ 🐷': [47.514029888841016,
  47.52803412162209,
  47.54197472310934,
  47.55585198242546,
  47.56966618737945,
  47.58341762447259,
  47.59710657890439,
  47.61073333457848,
  47.62429817410851,
  47.63780137882403,
  47.65124322877629,
  47.66462400274407,
  47.67794397823947,
  47.691203431513635,
  47.70440263756252,
  47.71754187013255,
  47.73062140172636,
  47.74364150360838,
  47.7566024458105,
  47.76950449713767,
  47.78234792517344,
  47.79513299628557,
  47.807859975631494,
  47.82052912716386,
  47.83314071363598,
  47.84569499660729,
  47.85819223644879,
  47.870632692348394,
  47.88301662231637,
  47.895344283190646,
  47.10585170816856],
 '10 แกงจืดหมูสับวุ้นเส้น': [57.01683586660922,
  57.03364094594651,
  57.05036966773121,
  57.06702237891055,
  57.083599424855336,
  57.100101149367106,
  57.11652789468526,
  57.13288000149417,
  57.14915780893021,
  57.165361654588835,
  57.181491874531545,
  57.19754880329288,
  57.21353277388736,
  57.229

In [34]:

conn.close()